In [2]:
#예제 4.3 가중치 초기화 함수(1)

from torch import nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1,2),
            nn.Sigmoid()
        )

        self.fc = nn.Linear(2,1)
        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.layer[0].weight)
        self.layer[0].bias.data.fill_(0.01)

        nn.init.xavier_uniform_(self.fc.weight)
        self.fc.bias.data.fill_(0.01)

model = Net()

In [3]:
#예제 4.4 가중치 초기화 함수(2)

from torch import nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1,2),
            nn.Sigmoid()            
        )
        self.fc = nn.Linear(2,1)
        self.apply(self._init_weights) #Net 클래스의 모든 모듈에 _init_weights 함수를 재귀적으로 적용한다.
        #여기서 모듈은 nn.Module을 상속받은 모든 요소를 의미하며, 이는 개별 레이어 뿐만 아니라 시퀀셜 자체와 전체 클래스 자체도 포함된다.
        #따라서 각 모듈이 _init_weights 함수에 의해 방문될 때마다 해당 모듈에 대한 정보가 출력된다.

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.constant_(module.bias, 0.01) #상수초기화 함수
        print(f"Apply : {module}")

model = Net()

Apply : Linear(in_features=1, out_features=2, bias=True)
Apply : Sigmoid()
Apply : Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Sigmoid()
)
Apply : Linear(in_features=2, out_features=1, bias=True)
Apply : Net(
  (layer): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): Sigmoid()
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
#예제 4.10 단어 삽입

import nlpaug.augmenter.word as naw

texts = ["Those who can imagine anything can create the impossible.",
         "We can only see a chort diatance ahead, but we can see plenty there that needs to be done.",
         "If a machine is expected to be infaillible, it cannot also be intelligent."]
         
aug = naw.ContextualWordEmbsAug(model_path = "bert-base-uncased", action = "insert")
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

src : Those who can imagine anything can create the impossible.
dst : those who can imagine sees anything that can create the impossible impossible.
------------------
src : We can only see a chort diatance ahead, but we can see plenty there that needs to be done.
dst : well we can only still see through a chort the diatance ahead, but we probably can see plenty ahead there so that needs to be done.
------------------
src : If a machine is expected to be infaillible, it cannot also be intelligent.
dst : imagine if a machine is expected to be mentally infaillible, but it also cannot automatically also be genetically intelligent.
------------------


In [7]:
import nlpaug.augmenter.word as naw

texts = ["Those who can imagine anything can create the impossible.",
         "We can only see a short diatance ahead, but we can see plenty there that needs to be done.",
         "If a machine is expected to be infaillible, it cannot also be intelligent."]
reserved_tokens = [["can","can't","cannot","could"],]        
 
reserved_aug = naw.ReservedAug(reserved_tokens = reserved_tokens)
augmented_texts = reserved_aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

src : Those who can imagine anything can create the impossible.
dst : Those who could imagine anything cannot create the impossible.
------------------
src : We can only see a short diatance ahead, but we can see plenty there that needs to be done.
dst : We cannot only see a short diatance ahead, but we could see plenty there that needs to be done.
------------------
src : If a machine is expected to be infaillible, it cannot also be intelligent.
dst : If a machine is expected to be infaillible, it could also be intelligent.
------------------


In [9]:
# 예제 4.16 통합 클래스 및 변환 적용 방식
from PIL import Image
from torchvision import transforms

transform = transforms.Compose(
    [
        transforms.Resize(size=(512,512)),
        transforms.ToTensor()
    ]
)

image = Image.open("datasets/images/cat.jpg")
transformed_image = transform(image)

print(transformed_image.shape)

torch.Size([3, 512, 512])


In [11]:
#예제 4.17 회전 및 대칭

transform = transforms.Compose([
    transforms.RandomRotation(degrees=30, expand=False, center=None),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=-0.5)
])

In [12]:
#예제 4.18 자르기 및 패딩
transform = transforms.Compose([
    transforms.RandomCrop(size=(512,512)),
    transforms.Pad(padding=50, fill=(127,127,255), padding_mode="constant")
]
)    

In [13]:
#예제 4.19 크기 조정
transform = transforms.Compose([
    transforms.Resize(size=(512,512))
])

In [14]:
#에제 4.20 아핀 변환
transform = transforms.Compose([
    transforms.RandomAffine(
        degrees = 15, translate=(0.2,0.2),
        scale=(0.8,1.2), shear=15
    )
])

In [15]:
#예제 4.21 색상 변환 및 정규화
transform = transforms.Compose([
    transforms.ColorJitter(
        brightness = 0.3, contrast = 0.3, saturation = 0.3, hue = 0.3
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = [0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225]
    ),
    transforms.ToPILImage()
])

In [17]:
#예제 4.22 노이즈 추가
import numpy as np
from PIL import Image
from torchvision import transforms
from imgaug import augmenters as iaa

class IaaTransforms:
    def __init__(self):
        self.seq = iaa.Sequential([
            iaa.SaltAndPepper(p=(0.03, 0.07)),
            iaa.Rain(speed=(0.3,0.7))
        ])

    def __call__(self, images):
        images = np.array(images)
        augmented = self.seq.augment_image(images)
        return Image.fromarray(augmented)
    
transform = transforms.Compose([
    IaaTransforms()
])

In [18]:
#예제 4.23 무작위 지우기
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomErasing(p=1.0,value=0), #컷아웃
    transforms.RandomErasing(p=1.0,value="random"), #무작위 지우기
    transforms.ToPILImage()
])

In [21]:
#예제 4.24 혼합
import numpy as numpy
from PIL import Image
from torchvision import transforms

class Mixup:
    def __init__(self, target, scale, alpha=0.5, beta=0.5):
        self.target = target
        self.scale = scale
        self.alpha = alpha
        self.beta = beta

    def __call__(self, image):
        image = np.array(image)
        target = self.target.resize(self.scale)
        target = np.array(target)
        mix_image = image*self.alpha+target*self.beta
        return Image.fromarray(mix_image.astype(np.unit8))
    
transform = transforms.Compose(
    [
        transforms.Resize((512,512)),
        Mixup(
            target=Image.open("/home/yeeun/NLPCV/datasets/images/dog.jpg"),
            scale=(512,512),
            alpha=0.5,
            beta=0.5
        )
    ]
)